In [1]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.aiohttp.transport import AiohttpTransport
from gremlin_python.process.traversal import *

In [2]:
PORT = 8182
SERVER = "database-1-instance-1.ccdtlakfhgyk.us-east-1.neptune.amazonaws.com"
ENDPOINT = f"https://{SERVER}:{PORT}/gremlin"

In [3]:
# connect to Neptune DB instance
graph=Graph()
connection = DriverRemoteConnection(ENDPOINT, 'g',
    transport_factory=lambda:AiohttpTransport(call_from_event_loop=True)
)
g = graph.traversal().withRemote(connection)

In [17]:
# Sample Query 1: Print names of companies
results = (g.V().hasLabel('Company').values('name').toList())
for index, company in enumerate(results):
    print(f"Company {index}: {company}")

Company 0: Example Corp
Company 1: AnyCompany


In [18]:
# Sample Query 2: Print first and last name of all persons
results = (g.V().hasLabel('Person').local(__.values('firstName','lastName').fold()).toList())
for index, person in enumerate(results):
    print(f"Person {index}: {person[0]} {person[1]}")

Person 0: Martha Rivera
Person 1: Richard Roe
Person 2: Li Juan
Person 3: John Stiles
Person 4: Saanvi Sarkar


In [19]:
# Sample Query 3: Print persons details and company they work for Example Corp (c-1)
# Eg. Martha Rivera works for Example Corp as Principal Analyst
results = g.V('c-1').bothE().otherV().limit(10).path().by(__.valueMap(True)).toList()
for data in results:
    company = data[0]['name'][0]
    role = data[1]['role']
    first_name = data[2]['firstName'][0]
    last_name = data[2]['lastName'][0]
    print(f"{first_name} {last_name} works for {company} as {role}.")

Martha Rivera works for Example Corp as Principal Analyst.
Richard Roe works for Example Corp as Senior Analyst.
Li Juan works for Example Corp as Analyst.
John Stiles works for Example Corp as Analyst.


In [16]:
# Sample Query 4: Add new record Tony Parker (p06) worked for AnyCompany (c-2)
g.addV('Person').property(id, 'p-6').property('firstName', 'Tony').property('lastName', 'Parker')
g.V('p-6').addE('worked_for').to(__.V('c-2')).property('role','Software Engineer')

[['V', 'p-6'], ['addE', 'worked_for'], ['to', [['V', 'c-2']]], ['property', 'role', 'Software Engineer'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [20]:
connection.close()